# Предсказание ухода клиента для банка "Бета-Банк"
## Постановка задачи

"Бета-Банк" решает проблему оттока клиентов. Согласно расчетов маркетологов банка, сохранять текущих клиентов дешевле, чем привлекать новых. 

Нужно научиться прогнозировать, уйдёт клиент из банка в ближайшее время или нет. Исходные данные для анализа - исторические данные о поведении клиентов и расторжении договоров с банком.

Нужно построить прогнозирующую модель с предельно большим значением F1-меры (не менее 0.59), затем проверить F1-меру на тестовой выборке. Дополнительно нужно измерять метрику AUC-ROC и сравнить её значения с F1-мерой.


## Импорт библиотек и данных
Импортируем датасет в переменную `df` и ознакомимся с данными. 

Исходные данные представляют собой датасет на 10_000 строк и 14 столбцов. Названия столбцов в CamelCase. Три столбца (`RowNumber`, `CustomerId`, `Surname`) неинформативны, их нужно будет удалить. Типы данных адекватны, но можно преобразовать колонки `HasCrCard` и `IsActiveMember` из численного типа в логический. В колонке `Tenure` есть пустые значения, от которых нужно будет избавиться.

В целом данные адекватны и пригодны для анализа после предобработки.

In [ ]:
from pathlib import Path

import pandas as pd
pd.set_option('display.float_format', '{:,.4f}'.format)

import matplotlib.pyplot as plt

import numpy as np
from sklearn.model_selection import train_test_split 
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler 
from sklearn.metrics import f1_score
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.utils import shuffle

In [ ]:
my_path = Path('/home/klarazetkin/Documents/yandex/module_2/project_3')
if my_path.is_dir():
    df = pd.read_csv('/home/klarazetkin/Documents/yandex/module_2/project_3/Churn.csv')
else:
    df = pd.read_csv('/datasets/Churn.csv')

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
df.isna().mean()

In [ ]:
df.info()

## Подготовка данных
### Переименование столбцов
Названия столбцов переведем из CamelCase в более привычный snake_case.

In [ ]:
df = df.rename(columns={"RowNumber": "row_number", "CustomerId": "customer_id", "Surname": "surname", "CreditScore": "credit_score", "Geography": "geography", "Gender": "gender", "Age": "age", "Tenure": "tenure", "Balance": "balance", "NumOfProducts": "number_of_products", "HasCrCard": "has_credit_card", "IsActiveMember": "is_active_member", "EstimatedSalary": "estimated_salary", "Exited": "exited"})

In [ ]:
df.columns

### Замена типов столбцов
Заменим тип столбцов `has_credit_card` и `is_active_member` на логический.

In [ ]:
df['has_credit_card'] = df['has_credit_card'].astype('bool')
df['is_active_member'] = df['is_active_member'].astype('bool')

### Удаление лишних столбцов
Скорей всего, информация об имении и id клиента, а также `row_number` ничего не даст модели и будет мешать обучению. Уберем эти столбцы.

In [ ]:
print(df.shape)
df = df.drop(['surname', 'customer_id', 'row_number'], axis=1)
print(df.shape)

### Заполнение NaN
Столбец `tenure` содержит пустые значения. Заполним их медианными.

In [ ]:
print(df['tenure'].isna().sum())
df['tenure'] = df['tenure'].fillna(df['tenure'].median())
print(df['tenure'].isna().sum())

### Кодирование категориальных признаков техникой OHE
В колонках `gender` и `geography` содержатся категориальные признаки. Кодируем их техникой One-Hot Encoding, т.к. она подходит для всех моделей.

In [ ]:
df.head()

In [ ]:
print(df['geography'].unique())
print(df['gender'].unique())

In [ ]:
df = pd.get_dummies(df, drop_first=True)

In [ ]:
df.shape

In [ ]:
df.columns

### Масштабирование данных
Численные значения в колонках `balance` и `estimated_salary` на два порядка превышают значения в других численных колонках. Чтоб модель не считала эти данные более важными, масштабируем численные данные.

In [ ]:
numeric = ['credit_score', 'age', 'tenure', 'balance', 'number_of_products', 'estimated_salary']

scaler = StandardScaler()
scaler.fit(df[numeric]) 

df[numeric] = scaler.transform(df[numeric])

In [ ]:
df.head()

### Выделение обучающей, валидационной и тестовой выборок
Разделим имеющийся датасет на три части: обучающую (60%), валидационную (20%) и тестовую (20%) выборки. Укажем параметр `random_state=666` для обеспечения повторяемости результата.

In [ ]:
df_train, df_valid_and_test = train_test_split(df, test_size=0.4, stratify=df['exited'], random_state=666)

In [ ]:
df_valid, df_test = train_test_split(df_valid_and_test, test_size=0.5, stratify=df_valid_and_test['exited'], random_state=666)

In [ ]:
# проверим размеры выборок
print(len(df_train))
print(len(df_valid))
print(len(df_test))

In [ ]:
# проверим, что % положительного класса целевого параметра представлены в выборках пропорционально
print(df_train['exited'].mean())
print(df_valid['exited'].mean())
print(df_test['exited'].mean())

### Определение целевого признака и других признаков
Целевой признак - факт ухода клиента, отраженный в колонке `df['exited']`.

In [ ]:
features_train = df_train.drop('exited', axis=1)
target_train = df_train['exited']
features_valid = df_valid.drop('exited', axis=1)
target_valid = df_valid['exited']
features_test = df_test.drop('exited', axis=1)
target_test = df_test['exited']

## Создание и обучение моделей предсказания

### Исследование баланса классов
Целевой признак - факт ухода клиента. Исследуем баланс классов. Соотношение позитивного и негативного классов приблизительно 1 : 4, что довольно далеко от баланса 1 : 1. Попробуем обучить модели без учета дисбаланса, проверим их результаты и при необходимости переобучим с учетом дисбаланса.

In [ ]:
df['exited'].value_counts()

In [ ]:
df['exited'].mean()

### Создание и обучение моделей без учета дисбаланса классов
#### Логистическая регрессия
Логистическая регрессия, обученная без учета дисбаланса классов, показала результат `f1 = 0.3309608540925267`. Значение меры `AUC-ROC = 0.7454167282490886`. Значение целевого признака очень далеко от минимально приемлемого `f1 = 0.59`.

Создадим хэш `metrics` с финальными значениями метрик, куда будем записывать все значения метрик обученных моделей.

In [ ]:
model = LogisticRegression(random_state=666, solver='liblinear')
model.fit(features_train, target_train)
predicted_valid = model.predict(features_valid)
f1_of_model = f1_score(target_valid, predicted_valid)
print('f1 =', f1_of_model)

In [ ]:
probabilities_valid = model.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]
auc_roc = roc_auc_score(target_valid, probabilities_one_valid)
print('AUC-ROC =', auc_roc)

In [ ]:
metrics = {'logistic_regression_unbalanced': {'f1': f1_of_model, 'auc_roc': auc_roc}}

#### Решающее дерево

У решающего дерева, обученного без учета дисбаланса классов, результат лучше: максимальное значение `f1 = 0.5796269727403156`и `auc_roc = 0.8185935806483398`. Лучшая модель имеет глубину 8 уровней.

In [ ]:
best_model = None
best_f1 = 0
best_auc_roc = 0
tree_metrics = []

for depth in range(2, 21):
    model = DecisionTreeClassifier(max_depth=depth, random_state=666)
    model.fit(features_train, target_train)
    predicted_valid = model.predict(features_valid)
    f1_of_model = f1_score(target_valid, predicted_valid)
    
    probabilities_valid = model.predict_proba(features_valid)
    probabilities_one_valid = probabilities_valid[:, 1]
    auc_roc_of_model = roc_auc_score(target_valid, probabilities_one_valid)
    
    print("max_depth =", depth, ": ", end='  ')
    print('f1 =', f1_of_model, end='  ') 
    print('auc_roc =', auc_roc_of_model)
    
    tree_metrics.append([depth, f1_of_model, auc_roc_of_model])
    if f1_of_model > best_f1:
        best_f1 = f1_of_model
        best_model = model
    if auc_roc_of_model > best_auc_roc:
        best_auc_roc = auc_roc_of_model
        

print()        
print('best_model:', best_model)
print('best_f1:', best_f1)
print('best_auc_roc:', best_auc_roc)

In [ ]:
metrics['tree_unbalanced'] = {'f1': best_f1,  'auc_roc': 0.8185935806483398}

#### Случайный лес
Без учета дисбаланса классов лучший результат получился у модели случайного леса с 20 деревьями и глубиной 15: 
`f1 = 0.564885496183206` и `auc_roc = 0.8324888843728446`. 

In [ ]:
best_depth = 1
best_model = None
best_f1 = 0
best_auc_roc = 0
forest_metrics = []

# Сначала обучим лес с 20 деревьями-оценщиками и найдем оптимальную глубину

for depth in range(2, 21):
    model = RandomForestClassifier(n_estimators=20, max_depth=depth, random_state=666)
    model.fit(features_train, target_train)
    predicted_valid = model.predict(features_valid)
    
    f1_of_model = f1_score(target_valid, predicted_valid)
    
    probabilities_valid = model.predict_proba(features_valid)
    probabilities_one_valid = probabilities_valid[:, 1]
    auc_roc_of_model = roc_auc_score(target_valid, probabilities_one_valid)
    
    print("max_depth =", depth, ": ", end='  ')
    print('f1 =', f1_of_model, end='  ') 
    print('auc_roc =', auc_roc_of_model)
    
    forest_metrics.append([depth, f1_of_model, auc_roc_of_model])
    if f1_of_model > best_f1:
        best_depth = depth
        best_model = model
        best_f1 = f1_of_model
    if auc_roc_of_model > best_auc_roc:
        best_auc_roc = auc_roc_of_model
    
print()        
print('best_depth:', best_depth)
print('best_model:', best_model)
print('best_f1:', best_f1)
print('best_auc_roc:', best_auc_roc)

In [ ]:
# Теперь подберем оптимальное количество деревьев-оценщиков
forest_metrics_2 = []
best_trees = 20

for trees in range (20, 250, 10):
    model = RandomForestClassifier(n_estimators = trees, 
                                  max_depth=best_depth, random_state=666)
    model.fit(features_train, target_train)
    predicted_valid = model.predict(features_valid)
    
    f1_of_model = f1_score(target_valid, predicted_valid)
    
    probabilities_valid = model.predict_proba(features_valid)
    probabilities_one_valid = probabilities_valid[:, 1]
    auc_roc_of_model = roc_auc_score(target_valid, probabilities_one_valid)
    
    print("trees =", trees, ": ", end='  ')
    print('f1 =', f1_of_model, end='  ') 
    print('auc_roc =', auc_roc_of_model)
    forest_metrics_2.append([trees, f1_of_model, auc_roc_of_model])
    if f1_of_model > best_f1:
        best_trees = trees
        best_model = model
        best_f1 = f1_of_model
    if auc_roc_of_model > best_auc_roc:
        best_auc_roc = auc_roc_of_model
        

        
print()    
print('best_depth:', best_depth)
print('best_trees:', best_trees)
print('best_model:', best_model)
print('best_f1:', best_f1)
print('best_auc_roc:', best_auc_roc)

In [ ]:
metrics['forest_unbalanced'] = {'f1': best_f1, 'auc_roc': 0.8324888843728446}

### Обучение моделей с учетом дисбаланса классов: весовая балансировка
Воспользуемся весовой балансировкой для снижения влияния дисбаланса классов. Укажем для моделей параметр `class_weight='balanced'`, обучим их и оценим результаты. 

#### Логистическая регрессия

Обучим модель логистической регрессии, указав в гиперпараметрах `class_weight='balanced'`. У новой модели `f1 = 0.4716981132075472` и `AUC-ROC = 0.7508960242388412`. 

Это выше, чем у модели, обученной без учета дисбаланса, но метрики все равно далеки от приемлемых.

In [ ]:
model = LogisticRegression(random_state=666, solver='liblinear', class_weight='balanced')
model.fit(features_train, target_train)
predicted_valid = model.predict(features_valid)
f1_of_model = f1_score(target_valid, predicted_valid)
print('f1 =', f1_of_model)

In [ ]:
probabilities_valid = model.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]
auc_roc = roc_auc_score(target_valid, probabilities_one_valid)
print('AUC-ROC =', auc_roc)

In [ ]:
metrics['logistic_regression_class_weight_balanced'] = {'f1': f1_of_model, 'auc_roc': auc_roc}

#### Решающее дерево
Обучим модель решающего дерева, указав в гиперпараметрах `class_weight='balanced'`. Наиболее удачная модель имеет глубину 6 уровней.

Эта модель показала лучшие результаты, по сравнению с моделью, обученной без учета дисбаланса классов: у новой модели `f1 = 0.5695970695970696` и `auc_roc = 0.8309500936052814`. Целевой показатель `f1` стал хуже, чем  у модели, обученной без учета дисбаланса (`f1 = 0.5796269727403156`).


In [ ]:
best_model = None
best_f1 = 0
best_auc_roc = 0
tree_metrics = []

for depth in range(2, 21):
    model = DecisionTreeClassifier(max_depth=depth, random_state=666, class_weight='balanced')
    model.fit(features_train, target_train)
    predicted_valid = model.predict(features_valid)
    f1_of_model = f1_score(target_valid, predicted_valid)
    
    probabilities_valid = model.predict_proba(features_valid)
    probabilities_one_valid = probabilities_valid[:, 1]
    auc_roc_of_model = roc_auc_score(target_valid, probabilities_one_valid)
    
    print("max_depth =", depth, ": ", end='  ')
    print('f1 =', f1_of_model, end='  ') 
    print('auc_roc =', auc_roc_of_model)
    
    tree_metrics.append([depth, f1_of_model, auc_roc_of_model])
    if f1_of_model > best_f1:
        best_f1 = f1_of_model
        best_model = model
    if auc_roc_of_model > best_auc_roc:
        best_auc_roc = auc_roc_of_model
        

print()        
print('best_model:', best_model)
print('best_f1:', best_f1)
print('best_auc_roc:', best_auc_roc)

In [ ]:
metrics['tree_class_weight_balanced'] = {'f1': best_f1, 'auc_roc': 0.8309500936052814}

In [ ]:
metrics

#### Случайный лес
Обучим модель случайного леса, указав в гиперпараметрах `class_weight='balanced'`. 

Лучше всего себя показала модель случайного леса с 8 уровнями глубины и 80 деревьями-оценщиками. Ее метрики `f1 = 0.6148571428571429`  и `auc_roc = 0.855629249187112` - значения лучше, чем у лучшей модели леса, обученной без учета дисбаланса. 

In [ ]:
best_depth = 1
best_model = None
best_f1 = 0
best_auc_roc = 0
forest_metrics = []

# Сначала обучим лес с 20 деревьями-оценщиками и найдем оптимальную глубину

for depth in range(2, 21):
    model = RandomForestClassifier(n_estimators=20, max_depth=depth, class_weight='balanced', random_state=666)
    model.fit(features_train, target_train)
    predicted_valid = model.predict(features_valid)
    
    f1_of_model = f1_score(target_valid, predicted_valid)
    
    probabilities_valid = model.predict_proba(features_valid)
    probabilities_one_valid = probabilities_valid[:, 1]
    auc_roc_of_model = roc_auc_score(target_valid, probabilities_one_valid)
    
    print("max_depth =", depth, ": ", end='  ')
    print('f1 =', f1_of_model, end='  ') 
    print('auc_roc =', auc_roc_of_model)
    
    forest_metrics.append([depth, f1_of_model, auc_roc_of_model])
    if f1_of_model > best_f1:
        best_depth = depth
        best_model = model
        best_f1 = f1_of_model
    if auc_roc_of_model > best_auc_roc:
        best_auc_roc = auc_roc_of_model
    
print()        
print('best_depth:', best_depth)
print('best_model:', best_model)
print('best_f1:', best_f1)
print('best_auc_roc:', best_auc_roc)

In [ ]:
# Теперь подберем оптимальное количество деревьев-оценщиков
forest_metrics_2 = []
best_trees = 1

for trees in range (20, 250, 10):
    model = RandomForestClassifier(n_estimators = trees, 
                                  max_depth=best_depth, class_weight='balanced', random_state=666)
    model.fit(features_train, target_train)
    predicted_valid = model.predict(features_valid)
    
    f1_of_model = f1_score(target_valid, predicted_valid)
    
    probabilities_valid = model.predict_proba(features_valid)
    probabilities_one_valid = probabilities_valid[:, 1]
    auc_roc_of_model = roc_auc_score(target_valid, probabilities_one_valid)
    
    print("trees =", trees, ": ", end='  ')
    print('f1 =', f1_of_model, end='  ') 
    print('auc_roc =', auc_roc_of_model)
    forest_metrics_2.append([trees, f1_of_model, auc_roc_of_model])
    if f1_of_model > best_f1:
        best_trees = trees
        best_model = model
        best_f1 = f1_of_model
    if auc_roc_of_model > best_auc_roc:
        best_auc_roc = auc_roc_of_model
        

        
print()    
print('best_depth:', best_depth)
print('best_trees:', best_trees)
print('best_model:', best_model)
print('best_f1:', best_f1)
print('best_auc_roc:', best_auc_roc)

In [ ]:
metrics['forest_class_weight_balanced'] = {'f1': best_f1, 'auc_roc': 0.855629249187112}

In [ ]:
metrics

### Обучение моделей с учетом дисбаланса классов: метод upsampling


#### Увеличение выборки методом upsampling
Воспользуемся альтернативным методом снижения влияния дисбаланса классов - увеличим выборку методом `upsampling`. Для этого определим функцию `upsample()` и передадим ей обучающую выборку. Баланс классов доведем примерно до 1 : 1.

In [ ]:
def upsample(features, target, repeat):
    features_zeros = features_train[target_train == 0]
    features_ones = features_train[target_train == 1]
    target_zeros = target_train[target_train == 0]
    target_ones = target_train[target_train == 1]
    
    features_upsampled = pd.concat([features_zeros] + [features_ones] * repeat)
    target_upsampled = pd.concat([target_zeros] + [target_ones] * repeat)
    features_upsampled, target_upsampled = shuffle(features_upsampled, target_upsampled, random_state=12345)
    
    return features_upsampled, target_upsampled
    

features_upsampled, target_upsampled = upsample(features_train, target_train, 4)

In [ ]:
print(target_upsampled.shape)
print(target_upsampled.mean())

#### Логистическая регрессия

Обучим модель логистической регрессии на увеличенной выборке. У новой модели `f1 = 0.4731914893617022` и `AUC-ROC = 0.750965304463494`. Эта модель показала лучшие результаты, по сравнению с моделью, обученной без учета дисбаланса классов, и с моделью, обученной с учетом весовой балансировки. Целевой показатель по-прежнему далек от приемлемого `f1 = 0.59`.

In [ ]:
model = LogisticRegression(random_state=666, solver='liblinear')
model.fit(features_upsampled, target_upsampled)
predicted_valid = model.predict(features_valid)
f1_of_model = f1_score(target_valid, predicted_valid)
print('f1 =', f1_of_model)

In [ ]:
probabilities_valid = model.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]
auc_roc = roc_auc_score(target_valid, probabilities_one_valid)
print('AUC-ROC =', auc_roc)

In [ ]:
metrics['logistic_regression_upsampled'] = {'f1': f1_of_model, 'auc_roc': auc_roc}
metrics

#### Решающее дерево

Обучим модель решающего дерева на увеличенной выборке. Наиболее удачная модель имеет глубину 6 уровней.

У новой модели `f1 = 0.5693296602387511` и  `auc_roc = 0.8294428638289487`. Целевой показатель `f1` немного хуже, чем у модели, обученной с весовой балансировкой (`f1 = 0.5695970695970696`), и значительно хуже, чем у модели, обученной без учета дисбаланса (`f1 = 0.5796269727403156`).


In [ ]:
best_model = None
best_f1 = 0
best_auc_roc = 0
tree_metrics = []

for depth in range(2, 21):
    model = DecisionTreeClassifier(max_depth=depth, random_state=666)
    model.fit(features_upsampled, target_upsampled)
              
    predicted_valid = model.predict(features_valid)
    f1_of_model = f1_score(target_valid, predicted_valid)
    
    probabilities_valid = model.predict_proba(features_valid)
    probabilities_one_valid = probabilities_valid[:, 1]
    auc_roc_of_model = roc_auc_score(target_valid, probabilities_one_valid)
    
    print("max_depth =", depth, ": ", end='  ')
    print('f1 =', f1_of_model, end='  ') 
    print('auc_roc =', auc_roc_of_model)
    
    tree_metrics.append([depth, f1_of_model, auc_roc_of_model])
    if f1_of_model > best_f1:
        best_f1 = f1_of_model
        best_model = model
    if auc_roc_of_model > best_auc_roc:
        best_auc_roc = auc_roc_of_model
        

print()        
print('best_model:', best_model)
print('best_f1:', best_f1)
print('best_auc_roc:', best_auc_roc)

In [ ]:
metrics['tree_upsampled'] = {'f1': best_f1, 'auc_roc': 0.8294428638289487}
metrics

#### Случайный лес
Обучим модель случайного леса на увеличенной выборке. Наиболее точной из обученных этим методом моделей является модель случайного леса с 8 уровнями глубины и 20 деревьями-оценщиками.  

Ее метрики `f1 = 0.607621009268795` и `auc_roc = 0.8568809119125038`. Значение `f1` хуже, чем у лучшей модели леса с весовой балансировкой (`f1 = 0.6148571428571429`); значение `auc_roc` новой модели лучше, но этот показатель не целевой.

In [ ]:
best_depth = 1
best_model = None
best_f1 = 0
best_auc_roc = 0
forest_metrics = []

# Сначала обучим лес с 20 деревьями-оценщиками и найдем оптимальную глубину

for depth in range(2, 21):
    model = RandomForestClassifier(n_estimators=20, max_depth=depth, random_state=666)
    model.fit(features_upsampled, target_upsampled)
    predicted_valid = model.predict(features_valid)
    
    f1_of_model = f1_score(target_valid, predicted_valid)
    
    probabilities_valid = model.predict_proba(features_valid)
    probabilities_one_valid = probabilities_valid[:, 1]
    auc_roc_of_model = roc_auc_score(target_valid, probabilities_one_valid)
    
    print("max_depth =", depth, ": ", end='  ')
    print('f1 =', f1_of_model, end='  ') 
    print('auc_roc =', auc_roc_of_model)
    
    forest_metrics.append([depth, f1_of_model, auc_roc_of_model])
    if f1_of_model > best_f1:
        best_depth = depth
        best_model = model
        best_f1 = f1_of_model
    if auc_roc_of_model > best_auc_roc:
        best_auc_roc = auc_roc_of_model
    
print()        
print('best_depth:', best_depth)
print('best_model:', best_model)
print('best_f1:', best_f1)
print('best_auc_roc:', best_auc_roc)

In [ ]:
# Теперь подберем оптимальное количество деревьев-оценщиков
forest_metrics_2 = []
best_trees = 20

for trees in range (20, 250, 10):
    model = RandomForestClassifier(n_estimators = trees, 
                                  max_depth=best_depth, random_state=666)
    model.fit(features_upsampled, target_upsampled)
    predicted_valid = model.predict(features_valid)
    
    f1_of_model = f1_score(target_valid, predicted_valid)
    
    probabilities_valid = model.predict_proba(features_valid)
    probabilities_one_valid = probabilities_valid[:, 1]
    auc_roc_of_model = roc_auc_score(target_valid, probabilities_one_valid)
    
    print("trees =", trees, ": ", end='  ')
    print('f1 =', f1_of_model, end='  ') 
    print('auc_roc =', auc_roc_of_model)
    forest_metrics_2.append([trees, f1_of_model, auc_roc_of_model])
    if f1_of_model > best_f1:
        best_trees = trees
        best_model = model
        best_f1 = f1_of_model
    if auc_roc_of_model > best_auc_roc:
        best_auc_roc = auc_roc_of_model
        

        
print()    
print('best_depth:', best_depth)
print('best_trees:', best_trees)
print('best_model:', best_model)
print('best_f1:', best_f1)
print('best_auc_roc:', best_auc_roc)

In [ ]:
metrics['forest_upsampled'] = {'f1': best_f1, 'auc_roc': 0.8568809119125038}
metrics

### Выбор и финальное тестирование лучшей модели
#### Выбор лучшей модели
Рассмотрим характеристики лучших из обученных каждым методом моделей и выберем самую лучшую. Лучший целевой показатель у модели случайного леса, обученного с учетом балансировки классов. Вот эта модель:
> RandomForestClassifier(bootstrap=True, class_weight='balanced',
                       criterion='gini', max_depth=8, max_features='auto',
                       max_leaf_nodes=None, min_impurity_decrease=0.0,
                       min_impurity_split=None, min_samples_leaf=1,
                       min_samples_split=2, min_weight_fraction_leaf=0.0,
                       n_estimators=80, n_jobs=None, oob_score=False,
                       random_state=666, verbose=0, warm_start=False)

In [ ]:
metrics

In [ ]:
best_f1 = 0
best_model_name = None

for item in metrics.items():
    if item[1]['f1'] > best_f1:
        best_f1 = item[1]['f1']
        best_model_name = item[0]
        
print(best_model_name, metrics[best_model_name])

#### Финальное тестирование
Проведем финальное тестирование лучшей модели - модели случайного леса:

> best_model: RandomForestClassifier(class_weight='balanced', max_depth=8, n_estimators=80, random_state=666)

Модель показала удовлетворительные результаты на тестовой и валидационной выборках:

f1 на обучающей выборке:  0.7182569496619083

f1 на валидационной выборке:  0.6148571428571429

f1 на тестовой выборке:  0.6131549609810478

AUC-ROC на обучающей выборке:  0.9313193174663743

AUC-ROC на валидационной выборке:  0.855629249187112

AUC-ROC на тестовой выборке:  0.8506395455547998

In [ ]:
best_model = RandomForestClassifier(n_estimators = 80, 
                                  max_depth=8, class_weight='balanced', random_state=666)
print(best_model)
best_model.fit(features_train, target_train)
predicted_train = best_model.predict(features_train) 
predicted_valid = best_model.predict(features_valid)
predicted_test = best_model.predict(features_test)
f1_train = f1_score(target_train, predicted_train)
f1_valid = f1_score(target_valid, predicted_valid)
f1_test = f1_score(target_test, predicted_test)


probabilities_train = best_model.predict_proba(features_train)
probabilities_one_train = probabilities_train[:, 1]
auc_roc_train = roc_auc_score(target_train, probabilities_one_train)

probabilities_valid = best_model.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]
auc_roc_valid = roc_auc_score(target_valid, probabilities_one_valid)

probabilities_test = best_model.predict_proba(features_test)
probabilities_one_test = probabilities_test[:, 1]
auc_roc_test = roc_auc_score(target_test, probabilities_one_test)

print("Наилучшая модель")
print("f1 на обучающей выборке: ", f1_train)
print("f1 на валидационной выборке: ", f1_valid)
print("f1 на тестовой выборке: ", f1_test)

print("AUC-ROC на обучающей выборке: ", auc_roc_train)
print("AUC-ROC на валидационной выборке: ", auc_roc_valid)
print("AUC-ROC на тестовой выборке: ", auc_roc_test)


In [ ]:
output_table = pd.DataFrame(
    [['Обучающая выборка', f1_train, auc_roc_train],
    ['Валидационная выборка', f1_valid, auc_roc_valid],
    ['Тестовая выборка', f1_test, auc_roc_test]],
    columns=['Выборка', 'F1-мера', 'AUC-ROC мера']
)
display(output_table)

## Вывод
Для решения задачи предсказания ухода клиента банка были построены и обучены модели трех типов: решающее дерево, случайный лес и логистическая регрессия. Был произведен подбор гиперпараметров моделей и выбрана лучшая модель для каждого типа. 

Модели были обучены сначала без учета дисбаланса классов в датасете, а после с учетом этого дисбаланса. Для того, чтоб исправить дисбаланс, были испробованы два метода: 

* обучающая выборка была увеличена методом `upsampling`, доля положительного (редкого) класса до ведена до 50%;
* модели были обучены с весовой балансировкой (при инициализации модели указан гиперпараметр `class_weight='balanced'`).

Модели, обученные с учетом дисбаланса, показали лучшие результаты в сравнении с моделями, обученными без учета дисбаланса классов, хотя апсамплинг улучшил целевой показатель только для логистической регрессии. 

Лучше всего себя показала модель решающего леса с гиперпараметрами: `max_depth=8`, `n_estimators=80`, `class_weight='balanced'`, обученная на валидационной выборке, не измененной методом `upsampling`.

Метрики этой модели приведены в таблице:

In [ ]:
display(output_table)

Модель рекомендуется дообучить и начать использовать в бизнесе.